# DSCI 525 Lab 1 
 
**Group 6: Crystal Geng, Tony Zoght, Chester Wang, HanChen Wang**


In this project, our objective is to get familiar with processing datasets and constructing machine learning models at scale. We will be working with a dataset containing daily precipitation data from New South Wales (NSW), Australia, covering the years from 1889 to 2014. The dataset is derived from CMIP6, a global cooperative initiative that brings together climate modeling outputs from various research groups.

The dataset comprises 6 columns, which represent timestamps, spatial data, and rainfall amounts (in mm/day). Our initial task (milestone 1) involves consolidating the modelled datasets into a single CSV file to facilitate subsequent processing. As the project progresses, we anticipate developing and deploying ensemble machine learning models in the cloud to forecast daily rainfall in Australia based on the dataset. The dataset features outputs from multiple climate models, with the actual rainfall observation as the target variable.

We have sourced the dataset from figshare, and our ultimate aim is to develop an ensemble model that leverages these outputs and compare its predictions with the actual rainfall data. By the end of the project, we aim to have a cloud-deployed machine learning model available for public use.

## 1. Team-work contract
rubric={correctness:10}

Similar to what you did in DSCI 522 and DSCI 524, create a teamwork contract. The contract should outline how you are committed to working together so that you are accountable to one another. Again, you may start with your team contract document from previous project courses and adapt it to your new team. It is a fairly personal document, and please do not push it into your public repositories. Instead, save it somewhere your team can easily share it, and you can share a link to it or a copy with us in your submission to Canvas to prove you did this.

The teamwork contract link will be appended in the Canvas Submission. Since it's a rather private documentation, we will not be including it here. 

## 2. Creating a repository and project structure 
rubric={mechanics:10}

1. Similar to previous project courses, create a public repository under [UBC-MDS org](https://github.com/UBC-MDS) for your project. 
2. Write a brief introduction of the project in the `README`. 
3. Create a folder called `notebooks` in the repository and create a notebook for this milestone in that folder.


The project GitHub repo can be accessed using the link: https://github.com/UBC-MDS/525-group-6

In [2]:
# Importing packages for analysis
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

## 3. Downloading the data 
rubric={correctness:10}

1. Download the data from [figshare](https://figshare.com/articles/dataset/Daily_rainfall_over_NSW_Australia/14096681) to your local computer using the [figshare API](https://docs.figshare.com) (you need to make use of `requests` library).

2. Extract the zip file, again programmatically, similar to how we did it in class. 

>  You can download the data and unzip it manually. But we learned about APIs, so we can do it in a reproducible way with the `requests` library, similar to how we [did it in class](https://pages.github.ubc.ca/MDS-2022-23/DSCI_525_web-cloud-comp_students/lectures/lecture1.html#using-rest-api-lab-lecture).

> There are 5 files in the figshare repo. The one we want is: `data.zip`

Nagivating to the location folder that you want to download the files to. Here we are using HanChen's notebook as an example. The code is adapted from the DSCI 525 lecture notes. 

In [2]:
## Change it to the location that you want to download your files to.
%mkdir /Users/hcwang/Documents/GitHub/Block6/figshareexp
%cd /Users/hcwang/Documents/GitHub/Block6/figshareexp

# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"

mkdir: /Users/hcwang/Documents/GitHub/Block6/figshareexp: File exists
/Users/hcwang/Documents/GitHub/Block6/figshareexp


Here we are sending a `GET` request to list the available files

In [3]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

Here we download the specific data file we will be using in this exercise. Uncomment the code below to begin downloading. 

In [4]:
# %%time
# files_to_dl = ["data.zip"]  # feel free to add other files here
# for file in files:
#     if file["name"] in files_to_dl:
#         os.makedirs(output_directory, exist_ok=True)
#         urlretrieve(file["download_url"], output_directory + file["name"])

Uncomment the code below to begin unzipping the downloaded zipped file. 

In [5]:
# %%time
# with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
#     f.extractall(output_directory)

Here is a list of available files in the local destination folder. 

In [6]:
%cd figsharerainfall/
%ls -ltr 

/Users/hcwang/Documents/GitHub/Block6/figshareexp/figsharerainfall
total 22326064
-rw-r--r--   1 hcwang  staff    95376895 Mar 27 14:28 MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
-rw-r--r--   1 hcwang  staff    94960113 Mar 27 14:28 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 hcwang  staff    82474546 Mar 27 14:28 NorESM2-LM_daily_rainfall_NSW.csv
-rw-r--r--   1 hcwang  staff   127613760 Mar 27 14:28 ACCESS-CM2_daily_rainfall_NSW.csv
-rw-r--r--   1 hcwang  staff   232118894 Mar 27 14:28 FGOALS-f3-L_daily_rainfall_NSW.csv
-rw-r--r--   1 hcwang  staff   330360682 Mar 27 14:28 CMCC-CM2-HR4_daily_rainfall_NSW.csv
-rw-r--r--   1 hcwang  staff   254009247 Mar 27 14:28 MRI-ESM2-0_daily_rainfall_NSW.csv
-rw-r--r--   1 hcwang  staff   235661418 Mar 27 14:28 GFDL-CM4_daily_rainfall_NSW.csv
-rw-r--r--   1 hcwang  staff   294260911 Mar 27 14:28 BCC-CSM2-MR_daily_rainfall_NSW.csv
-rw-r--r--   1 hcwang  staff   295768615 Mar 27 14:28 EC-Earth3-Veg-LR_daily_rainfall_NSW.csv
-rw-r--r--   1 hcwang

## 4. Combining data CSVs
rubric={correctness:10,reasoning:10}

1. Combine data CSVs into a single CSV using pandas.
    
2. When combining the CSV files, add an extra column called "model" that identifies the model.
    Tip 1: you can get this column populated from the file name, eg: for file name "SAM0-UNICON_daily_rainfall_NSW.csv", the model name is SAM0-UNICON
    Tip 2: Remember how we added "year" column when we combined airline CSVs. Here the regex will be to get word before an underscore ie, "/([^_]*)"

> Note: There is a file called `observed_daily_rainfall_SYD.csv` in the data folder that you downloaded. Make sure you exclude this file (programmatically or just take out that file from the folder) before you combine CSVs. We will use this file in our next milestone.

3. ***Compare*** run times on different machines within your team and summarize your observations. 

> Warning: Some of you might not be able to do it on your laptop. It's fine if you're unable to do it. Just make sure you discuss the reasons why you might not have been able to run this on your laptop. 

We are iterating through all csv files in the local directory, excluding the `observed_daily_rainfall_SYD.csv` file and the output file `daily_rainfall.csv`.  

We checked that the csvs have the same number of columns so we can simply concatenate the dataframes. The merged dataframe `daily_rainfall_df` will also include a column indicating its source (`model` column). 

In [7]:
%%time
daily_rainfall_df = pd.DataFrame()
for file in os.listdir("."):
    if file[-3:] == 'csv' and file != 'observed_daily_rainfall_SYD.csv' and file != 'daily_rainfall.csv':
        print(file)
        data = pd.read_csv(file)
        data['model'] = file.split("_")[0]
        daily_rainfall_df = pd.concat([daily_rainfall_df, data])

MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv
AWI-ESM-1-1-LR_daily_rainfall_NSW.csv
NorESM2-LM_daily_rainfall_NSW.csv
ACCESS-CM2_daily_rainfall_NSW.csv
FGOALS-f3-L_daily_rainfall_NSW.csv
CMCC-CM2-HR4_daily_rainfall_NSW.csv
MRI-ESM2-0_daily_rainfall_NSW.csv
GFDL-CM4_daily_rainfall_NSW.csv
BCC-CSM2-MR_daily_rainfall_NSW.csv
EC-Earth3-Veg-LR_daily_rainfall_NSW.csv
CMCC-ESM2_daily_rainfall_NSW.csv
NESM3_daily_rainfall_NSW.csv
MPI-ESM1-2-LR_daily_rainfall_NSW.csv
ACCESS-ESM1-5_daily_rainfall_NSW.csv
FGOALS-g3_daily_rainfall_NSW.csv
INM-CM4-8_daily_rainfall_NSW.csv
MPI-ESM1-2-HR_daily_rainfall_NSW.csv
TaiESM1_daily_rainfall_NSW.csv
NorESM2-MM_daily_rainfall_NSW.csv
CMCC-CM2-SR5_daily_rainfall_NSW.csv
KIOST-ESM_daily_rainfall_NSW.csv
INM-CM5-0_daily_rainfall_NSW.csv
MIROC6_daily_rainfall_NSW.csv
BCC-ESM1_daily_rainfall_NSW.csv
GFDL-ESM4_daily_rainfall_NSW.csv
CanESM5_daily_rainfall_NSW.csv
SAM0-UNICON_daily_rainfall_NSW.csv
CPU times: user 32.1 s, sys: 25.8 s, total: 57.9 s
Wall time: 1min 6s


Checking the merged dataframe and confirmed with lab members that the row and column numbers are correct. 

In [15]:
daily_rainfall_df.shape

(62467843, 7)

Writing the merged dataframe to csv. 

In [9]:
%%time
daily_rainfall_df.to_csv("daily_rainfall.csv", index=False)

CPU times: user 2min 55s, sys: 7.02 s, total: 3min 2s
Wall time: 3min 3s


In [18]:
%%sh
cksum daily_rainfall.csv

3747499371 6004371870 daily_rainfall.csv


### 4.3 Compare run times on different machines within your team and summarize your observations.
| Team Member | Operating System | RAM | Processor | Is SSD | Time to merge dataset| Time to write to a new csv|
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|:--:|
| HanChen Wang| OSX 13.2.1 | 8GB |Apple M2 (8 Cores)|Yes| 1m6s |3m3s|
| Tony Zoght  | OSX 12.5   | 32GB  |  Apple M1 (10 Cores         | Yes |    ||
| Chester Wang    | OSX 12.5                 |8GB     |2.9 GHz Dual-Core Intel Core i5           |Yes       |            ||
| Crystal Geng   |  OSX 12.6      | 8GB    | Apple M2 (8 Cores)       |   Yes     |     29.5s       | 2min45s|

`[NEED TO write more description on this]`

## 5. Load the combined CSV to memory and perform a simple EDA
rubric={correctness:10,reasoning:10}

1. Investigate at least two of the following approaches to reduce memory usage while performing the EDA (e.g., value_counts). Refer to lecture notes [here](https://pages.github.ubc.ca/MDS-2022-23/DSCI_525_web-cloud-comp_students/lectures/lecture1.html#some-tactics-to-deal-with-memory-issue).
    - Changing `dtype` of your data
    - Load just columns that we want
    - Loading in chunks
    
2. ***Compare*** run times on different machines within your team and summarize your observations.

Here we are performed the data loading using three techniques discussed in the DSCI 525 Lecture 1, which are:
1. Loading the entire table all at once (the slowest method).  
2. Loading the specific column of interest only. 
3. Laoding the csv in chunks. 

We will then perform a simple EDA on the number of observations for each model (`.value_counts` on the `model` column).

Before starting with the data loading, We first measured the speed to perform the simply EDA using preloaded dataframe above. This shows that the simple EDA step `does not` take a significant amount of time to complete (~ 1-2 seconds). Thus, we can use this to calculate the time each method takes to load the data. 

In [10]:
%%time
daily_rainfall_df['model'].value_counts()

CPU times: user 1.14 s, sys: 85.2 ms, total: 1.22 s
Wall time: 1.28 s


MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64

### 5.1 Loading the entire table all at once (the slowest method).  
This method takes from 30-106 seconds to complete depending on the computer we have. We will use this as the reference baseline when comparing with the other methods. 

In [11]:
%%time
# Reading all at once
df = pd.read_csv("daily_rainfall.csv")
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
CPU times: user 27.8 s, sys: 6.29 s, total: 34 s
Wall time: 36 s


### 5.2 Loading the specific column of interest only.  
This method takes less than half of the time compared to 5.1 on all our computers. This can be attributed to us pre-defining the column to read and thus it requires less time and memory to load the data. 

In [12]:
%%time
# Reading specific column only
use_cols = ['model']
df = pd.read_csv("daily_rainfall.csv",usecols=use_cols)
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
CMCC-CM2-HR4        3541230
CMCC-ESM2           3541230
CMCC-CM2-SR5        3541230
NorESM2-MM          3541230
TaiESM1             3541230
SAM0-UNICON         3541153
GFDL-ESM4           3219300
FGOALS-f3-L         3219300
GFDL-CM4            3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
AWI-ESM-1-1-LR       966420
MPI-ESM1-2-LR        966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
CPU times: user 15.4 s, sys: 1.52 s, total: 17 s
Wall time: 17.1 s


### 5.3 Laoding the csv in chunks. 
This method takes slightly less time than the method in 5.1 on all our computers. However, we have the better method in 5.2. 

In [13]:
%%time
# Reading in chucks
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("daily_rainfall.csv", chunksize=10_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
dtype: int64
CPU times: user 28.1 s, sys: 3.96 s, total: 32 s
Wall time: 22min 26s


### 5.4 Compare run times on different machines within your team and summarize your observations.
From the comparison table below, it can be seen that the time taken to read data (for all three tasks) are fairly consistent between Apple M1 and Apple M2 computers, with the speed in all three tasks slightly faster with Tony's computer primarily due to the 32GB RAM and 10 Cores processor. The 2.9GHz Dual-Core Intel Core i5 took the longest time for all three tasks. 

`please add more`

### Extra: storing the dataframe with different dataTypes  

We experimented storing the loaded dataframe in `float64` and `float32` and it's apparent the `float32` format conserves memory usage. 

In [14]:
print(f"Memory usage with float64: {daily_rainfall_df.memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float32: {daily_rainfall_df.astype('float32', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64: 3997.94 MB
Memory usage with float32: 2748.59 MB


| Team Member | Operating System | RAM | Processor | Is SSD | Time taken to read all at once| Time taken to read model column only| Time taken to read in chunks|
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|:----------:|:----------:|
| HanChen Wang| OSX 13.2.1 | 8GB |Apple M2 (8 Cores)|Yes| 37.1s|17.6s|33.6s|
| Tony Zoght  | OSX 12.5   | 32GB  |  Apple M1 (10 Cores)         | Yes |  30.1s  |   14.3s    |   29.5s  |
| Chester Wang    | OSX 12.5                 |8GB     |2.9 GHz Dual-Core Intel Core i5           |Yes       |109s            |48.2s            |94s            |
| Crystal Geng   |  OSX 12.6      | 8GB    | Apple M2 (8 Cores)       |   Yes     |  36s          | 15.8s           |   33.4s   |

## 6. Perform a simple EDA in R
rubric={correctness:15,reasoning:10}

Choose one of the methods listed below for transferring the dataframe (i.e., the entire dataset) from Python to R, and explain why you opted for this approach instead of the others.
- Parquet file
- Pandas exchange
- Arrow exchange

Once you have the dataframe in R, perform a simple EDA.

We choose to use Arrow for transferring the dataframe from Python to R because it shows enhanced performance and efficiency when we transfer between Python to R compared to Pandas exchange. This is primarily due to the fact that the time spent on serialization/deserialization process using Pandas is very long, while it only took 512 ms to finish the transferring using an arrow table. Although Parquet is another option that is more efficient than Pandas, Parquet requires additional on-disk storage since it would need to convert the csv file to Parquet files first. Therefore, using Arrow exchange might be more efficient and less time-consuming in our scenario since we are aiming for data exchange between Python and R. 

In [1]:
import pyarrow.dataset as ds
import pyarrow as pa
import pandas as pd
import pyarrow 
from pyarrow import csv
import rpy2_arrow.pyarrow_rarrow as pyra

In [4]:
import os
os.environ['R_HOME'] = '/Users/Jialing/opt/miniconda3/envs/525_2023/lib/R'

In [2]:
%load_ext rpy2.ipython

In [3]:
# Change it to the directory where your csv file is located 
filepathcsv = "/Users/Jialing/525/lab1/combined_data.csv"

In [4]:
%%time
dataset = ds.dataset(filepathcsv, format="csv")

table = dataset.to_table()

r_table = pyra.converter.py2rpy(table)

CPU times: user 11.1 s, sys: 1.24 s, total: 12.4 s
Wall time: 11.6 s


In [6]:
%%time
%%R -i r_table
start_time <- Sys.time()
suppressMessages(library(dplyr))
result <- r_table |> count(model) |> collect()
end_time <- Sys.time()
print(result %>% collect())
print(end_time - start_time)
print(result)


# A tibble: 27 × 2
   model                  n
   <chr>              <int>
 1 MPI-ESM-1-2-HAM   966420
 2 AWI-ESM-1-1-LR    966420
 3 NorESM2-LM        919800
 4 ACCESS-CM2       1932840
 5 FGOALS-f3-L      3219300
 6 CMCC-CM2-HR4     3541230
 7 MRI-ESM2-0       3037320
 8 GFDL-CM4         3219300
 9 BCC-CSM2-MR      3035340
10 EC-Earth3-Veg-LR 3037320
# ℹ 17 more rows
# ℹ Use `print(n = ...)` to see more rows
Time difference of 0.378994 secs
# A tibble: 27 × 2
   model                  n
   <chr>              <int>
 1 MPI-ESM-1-2-HAM   966420
 2 AWI-ESM-1-1-LR    966420
 3 NorESM2-LM        919800
 4 ACCESS-CM2       1932840
 5 FGOALS-f3-L      3219300
 6 CMCC-CM2-HR4     3541230
 7 MRI-ESM2-0       3037320
 8 GFDL-CM4         3219300
 9 BCC-CSM2-MR      3035340
10 EC-Earth3-Veg-LR 3037320
# ℹ 17 more rows
# ℹ Use `print(n = ...)` to see more rows
CPU times: user 1.22 s, sys: 575 ms, total: 1.79 s
Wall time: 512 ms
